In [6]:
#首先实现一个父类Node，然后基于这个父类依次实现Input Linear Sigmoid等模块。
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

#父类
class Node(object):
    #初始化
    def __init__(self,inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        self.value= None
        self.outbound_nodes = []
        
        self.gradients = {}
        
        for node in inbound_nodes:
            node.outbound_nodes.append(self)
        
    #前向传播
    def forward(self):
        raise NotImplementedError
        
    #反向传播
    def backward(self):
        raise NotImplementedError
        
class Input(Node):
    def __init__(self):
        Node.__init__(self)
        
    def forward(self):
        pass
    
    def backward(self):
        self.gradients={self:0}
        for n in self.outbound_nodes:
                self.gradients[self] += n.gradoents[self]
                
class Linear(Node):
    def __init__(self,X,W,b):
        Node.__init__(self,[X,W,b])
        
    def forward(self):
        X =self.inbound_bodes[0].value
        W =self.inbound_bodes[1].value
        b =self.inbound_bodes[2].value
        self.value = np.dot(X,W)+b
        
    def backward(self):
        self.gradients = {n:np.zeros_like(n.value) for n in self.inbound_nodes}
        for n in self.outbound_nodes:
            grad_cost = n.gradients[self]
            self.gradients[self.inbound_nodes[0]] += np.dot(grad_cost,self.inbound_nodes[1].value.T)
            self.fradients[self.inbound_nodes[1]] += np.dot(self.inbound_nodes[0].value.T,grad_cost)
            self.gradients[self.inbound_nodes[2]] +=np.sum(grad_cost,axis=0,keepdims=False)
            
class Sigmoid(Node):
    def __init__(self,node):
        Node.__init__(self,[node])
        
    def _sigmoid(self,x):
        return 1. / (1. + np.exp(-x))
    
    def forward(self):
        input_value = self.inbound_nodes[0].value
        self.value = self._sigmoid(input_value)
        
    def backward(self):
        self.gradients = {n:np.zeros_like(n.value) for n in self.inbound_nodes}
        for n in self.oubound_nodes:
            grad_cost = n.gradients[self]
            sigmoid = self.value
            self.gradients[self.inbound_nodes[0]] += sigmoid * (1 - sigmoid) * grad_cost
            
class MSE(Node):
    def __init__(self,y,a):
        Node.__init__(self,y,a)
        
    def forward(self):
        y = self.inbound_nodes[0].value.reshape(-1,1)
        a = self.inbound_nodes[1].value.reshape(-1,1)
        
        self.m = self.inbound_nodes[0].value.shape[0]
        self.diff = y-a
        self.value = np.mean(self.diff**2)
        
    def backward(self):
        self.gradients[self.inbound_nodes[0]] = (2 / self.m) * self.diff
        self.gradients[self.inbound_nodes[1]] = (-2 / self.m) * self.diff
        
def topological_sort(feed_dict):
    input_nodes = [n for n in feed.dict.keys()]
    G ={}
    nodes = [n for n in input_nodes]
    while len(nodes) >0 :
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in':set(),'out':set()}
        for m in n.outbound_nodes:
            if m not in G:
                G[m] = {'in':set(),'out':set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)
            
    L = []
    S=set(input_nodes)
    while len(S)>0:
        n = S.pop()
        if isinstance(n,Input):
            n.value = feed_dict[n]
            
        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']==0):
                S.add[m]
    return L

def forward_and_backward(graph):
    for n in graph:
        n.forward()
        
    for n in graph[::-1]:
        n.backward()
        

def sgd_update(trainables,learning_rate=1e-2):
    for t in trainables:
        t.value = t.value - learning_rate * t.gradients[t]
        

In [11]:
from sklearn.utils import resample
from sklearn import datasets

%matplotlib inline

data = datasets.load_iris()
X_=data.data
Y_=data.target
Y_[Y_==2] = 1#0是鸢尾花，1不是鸢尾花

np.random.seed(0)
n_features = X_.shape[1]#鸢尾花的特征数
n_class = 1#1个输出层
n_hidden =3 #1个输入，2个隐藏

X，y=Input(),Input()
W1,b1=Input(),Input()
W2,b2=Input(),Input()

l1=Linear(X,W1,b1)
s1=Sigmoid(l1)
l2=Linear(s1,W2,b2)
t1=Sigmoid(l2)
cost=MSE(y,t1)

SyntaxError: invalid character in identifier (<ipython-input-11-35bd62783fb6>, line 16)